<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.2
Pandas:  1.1.3


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...

  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     

  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                        

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [4]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv')

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [5]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [18]:
#ANSWER
df.iloc[:,0:5]

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [19]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-04-09T03:57:59.010Z,32.779333,-115.4350,11.64,2.69,ml,19.0,74.0,0.02775,0.25,...,2021-04-09T04:01:51.657Z,"6km SW of Holtville, CA",earthquake,0.53,0.71,0.168,29.0,automatic,ci,ci
1,2021-04-09T03:57:13.140Z,32.775333,-115.4335,11.66,2.81,ml,27.0,48.0,0.03193,0.30,...,2021-04-09T04:01:08.758Z,"6km SW of Holtville, CA",earthquake,0.50,0.80,0.246,27.0,automatic,ci,ci
2,2021-04-09T03:31:46.890Z,32.778000,-115.4390,12.12,2.78,ml,24.0,48.0,0.02802,0.29,...,2021-04-09T03:55:17.040Z,"7km SW of Holtville, CA",earthquake,0.52,1.02,0.233,26.0,automatic,ci,ci
3,2021-04-09T03:29:31.298Z,58.675000,-153.3582,4.00,2.60,ml,NaN,NaN,NaN,1.03,...,2021-04-09T03:35:16.892Z,"73 km NNW of Aleneva, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
4,2021-04-09T03:23:15.044Z,58.711500,-153.5527,0.00,2.60,ml,NaN,NaN,NaN,0.86,...,2021-04-09T03:36:57.242Z,"81 km NNW of Aleneva, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [21]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'
df = pd.read_html(url)
df

[                             Bank Name               City State   Cert  \
 0                    Almena State Bank             Almena    KS  15426   
 1           First City Bank of Florida  Fort Walton Beach    FL  16748   
 2                 The First State Bank      Barboursville    WV  14361   
 3                   Ericson State Bank            Ericson    NE  18265   
 4     City National Bank of New Jersey             Newark    NJ  21111   
 ..                                 ...                ...   ...    ...   
 558                 Superior Bank, FSB           Hinsdale    IL  32646   
 559                Malta National Bank              Malta    OH   6629   
 560    First Alliance Bank & Trust Co.         Manchester    NH  34264   
 561  National State Bank of Metropolis         Metropolis    IL   3815   
 562                   Bank of Honolulu           Honolulu    HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                            Equit

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [22]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [ ]:
#ANSWER
bikes.shape

What are the column names in `bikes`?

In [ ]:
#ANSWER
bikes.columns

What are the data types of these columns?

In [23]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [24]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [25]:
#ANSWER
print(type(bikes['season']))
print(type(bikes[['season']]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


How would we use object notation to show the first 4 rows of `atemp`?

In [26]:
#ANSWER
bikes['atemp'][:4]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [29]:
#E: bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: (0, 0)

What is the correct way to access the 1st row of the DataFrame by its index?

In [33]:
#ANSWER
bikes.iloc[0,:]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [34]:
#ANSWER
bikes.iloc[:,1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [37]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [40]:
#ANSWER
bikes.isna().sum().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [42]:
#ANSWER
bikes.isnull().sum() > 0

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [6]:
#ANSWER
def my_div(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        raise Exception('Cannot divide by zero!')
        
my_div(8,0)

Exception: Cannot divide by zero!

Apply the Pandas `isna` function to the following data objects:

In [54]:
x = 2.3
y = np.nan
print(x, y)

False True


In [55]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

False
True


In [57]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [58]:
#ANSWER
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER:
It specifies entries to be considered as NaN

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [59]:
#ANSWER
len(set(bikes.season))

4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [94]:
#ANSWER
b_dt = pd.to_datetime(bikes.dteday)
bikes['dteday'] = b_dt

def my_range(series):
    my_min = series.min()
    my_max = series.max()
    return my_max - my_min

print(my_range(bikes.instant))
print(my_range(bikes.dteday))
print(my_range(bikes.windspeed))

index = (bikes.dtypes == np.int64)|(bikes.dtypes == np.float64)

bikes_numeric = bikes.loc[:, index]

17378
730 days 00:00:00
0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [95]:
bikes_min, bikes_max = (min(bikes_numeric.min()), max(bikes_numeric.max()))
print(bikes_min, bikes_max)

0.0 17379.0


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [ ]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [ ]:
#ANSWER


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [96]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [97]:
#ANSWER
print(type(atemp_level))
print(atemp_level.head())

<class 'pandas.core.series.Series'>
0    (0.25, 0.5]
1    (0.25, 0.5]
2    (0.25, 0.5]
3    (0.25, 0.5]
4    (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]


Here is a random sample of `atemp_level`:

In [98]:
atemp_level.sample(5)          

1073        (0.25, 0.5]
8742     (-0.001, 0.25]
14000       (0.5, 0.75]
13479       (0.5, 0.75]
615      (-0.001, 0.25]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [99]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

3189     warm
16667    cool
10512    mild
1889     mild
11312    warm
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [105]:
#ANSWER
bikes['atemp_level'] = atemp_level

index = (bikes.atemp_level == 'mild')&(bikes.season == 2)

sum(index)

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [106]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [110]:
# Cleaning Data
print(f'{a}\n{b}\n{c}\n{d}\n{e}')

[0. 0. 0.]
[[1. 1. 1.]]
[[[7 6 6 9]
  [6 6 4 5]
  [2 6 7 4]]

 [[3 7 8 9]
  [8 9 6 8]
  [7 9 7 9]]]
[0 1 2 3]
[[1 2 3 4]
 [5 6 7 8]]


## Load Data

Load rock.csv and clean the dataset.

In [7]:
rock = pd.read_csv('rock.csv', na_values=['SONGFACTS.COM'])
rock.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1,1,1,1


## Check Column Names

Check column names and clean.

In [8]:
print(rock.dtypes)

Song Clean       object
ARTIST CLEAN     object
Release Year    float64
COMBINED         object
First?            int64
Year?             int64
PlayCount         int64
F*G               int64
dtype: object


# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [167]:
rock = rock.fillna(0)

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [135]:
#Fixed by entering na_values = 'SONGFACTS.COM when entering data'

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object


ValueError: Unable to parse string "SONGFACTS.COM" at position 1504

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [151]:
rock['First?'] = rock['First?'].astype(bool)
rock['Year?']  = rock['Year?'].astype(bool)

print(rock.min())
print(rock.max())

rock.loc[rock['Release Year']==1071, :]

rock.loc[547,'Release Year'] = 1971

Song Clean                     #9 Dream
ARTIST CLEAN                .38 Special
Release Year                       1955
COMBINED        #9 Dream by John Lennon
First?                             True
Year?                             False
PlayCount                             0
F*G                                   0
dtype: object
Song Clean                     Ziggy Stardust
ARTIST CLEAN                             a-ha
Release Year                             2014
COMBINED        Ziggy Stardust by David Bowie
First?                                   True
Year?                                    True
PlayCount                                 142
F*G                                       142
dtype: object


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [173]:
def info(series):
    song = series['Song Clean']
    artist = series['ARTIST CLEAN']
    
    if series['Release Year']<1970:
        period = 'before'
    else:
        period = 'after'
    return print(f'Song: {song}\nArtist: {artist}\nReleased {period} 1970\n')

def df_info(df, num):
    return info(df.iloc[num, :])

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [168]:
def float_or_nan(val):
    try:
        return float(val)
    except ValueError:
        return np.nan

def df_float(df):
    return df.applymap(float_or_nan)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0.0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1,1,1,1


## Apply these functions to your dataset

In [175]:
df_info(rock,0)

rock.apply(info, axis=1)

Song: Caught Up in You
Artist: .38 Special
Released after 1970

Song: Caught Up in You
Artist: .38 Special
Released after 1970

Song: Fantasy Girl
Artist: .38 Special
Released before 1970

Song: Hold On Loosely
Artist: .38 Special
Released after 1970

Song: Rockin' Into the Night
Artist: .38 Special
Released after 1970

Song: Art For Arts Sake
Artist: 10cc
Released after 1970

Song: Kryptonite
Artist: 3 Doors Down
Released after 1970

Song: Loser
Artist: 3 Doors Down
Released after 1970

Song: When I'm Gone
Artist: 3 Doors Down
Released after 1970

Song: What's Up?
Artist: 4 Non Blondes
Released after 1970

Song: Take On Me
Artist: a-ha
Released after 1970

Song: Baby, Please Don't Go
Artist: AC/DC
Released before 1970

Song: Back In Black
Artist: AC/DC
Released after 1970

Song: Big Gun
Artist: AC/DC
Released after 1970

Song: CAN'T STOP ROCK'N'ROLL
Artist: AC/DC
Released before 1970

Song: Dirty Deeds Done Dirt Cheap
Artist: AC/DC
Released after 1970

Song: For Those About To Rock
Ar

Song: Mercury Blues
Artist: Steve Miller Band
Released before 1970

Song: Rock 'N Me
Artist: Steve Miller Band
Released after 1970

Song: Space Cowboy
Artist: Steve Miller Band
Released before 1970

Song: Swingtown
Artist: Steve Miller Band
Released before 1970

Song: Take the Money and Run
Artist: Steve Miller Band
Released after 1970

Song: The Joker
Artist: Steve Miller Band
Released after 1970

Song: The Stake
Artist: Steve Miller Band
Released before 1970

Song: Threshold/Jet Airliner
Artist: Steve Miller Band
Released before 1970

Song: True Fine Love
Artist: Steve Miller Band
Released before 1970

Song: Your Cash Ain't Nothin But Trash
Artist: Steve Miller Band
Released before 1970

Song: Oh Sherrie
Artist: Steve Perry
Released after 1970

Song: Back In the High Life Again
Artist: Steve Winwood
Released after 1970

Song: Higher Love
Artist: Steve Winwood
Released after 1970

Song: Roll With It
Artist: Steve Winwood
Released after 1970

Song: While You See a Chance
Artist: Steve 

0       None
1       None
2       None
3       None
4       None
        ... 
2225    None
2226    None
2227    None
2228    None
2229    None
Length: 2230, dtype: object

In [176]:
df_float(rock)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0
1,NaN,NaN,0.0,NaN,1.0,0.0,3.0,0.0
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,0.0,NaN,1.0,0.0,1.0,0.0
2226,NaN,NaN,1981.0,NaN,1.0,1.0,32.0,32.0
2227,NaN,NaN,1975.0,NaN,1.0,1.0,109.0,109.0
2228,NaN,NaN,1983.0,NaN,1.0,1.0,1.0,1.0


## Describe the new float-only DataFrame.

In [189]:
rock_float = df_float(rock)
rock_float = rock_float.dropna(axis=1)
rock_float.describe()

,Release Year,First?,Year?,PlayCount,F*G
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1465.331390,1.0,0.741256,16.872646,15.048430
std,867.196161,0.0,0.438043,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



